In [1]:
from PIL import Image
import numpy as np
import shutil
import matplotlib.pyplot as plt
import os
import torch.nn as nn
from torch.autograd import Variable
import torchvision
from torchvision import models,transforms,datasets
import torch
#import bcolz
import time
%matplotlib inline
import pandas as pd
import random
import torch.nn.functional as F

In [5]:
os.chdir("Projet")

In [6]:
os.chdir('UTKFace')

In [7]:
def extract_label_str(wordtest,rate):
    suffix=".jpg.chip.jpg"
    wordtest=wordtest[:-len(suffix)]
    lstr=wordtest.split("_")
    age=lstr[0]
    sex="male"
    if lstr[1]==str(1):
        sex="female"
    nb=lstr[2]
    race='white'
    if (nb==str(1)):
        race='black'
    if (nb==str(2)):
        race='asian'
    if (nb==str(3)):
        race='indian'
    if (nb==str(4)):
        race='others'        
    val="train"
    x=random.uniform(0, 1)
    if x<rate:
        val="valid"
    return(age,sex,race,val)

In [8]:
def extract_labels_int(wordtest) :
    suffix=".jpg.chip.jpg"
    wordtest=wordtest[:-len(suffix)]
    lstr=wordtest.split("_")
    age=int(lstr[0])
    sex=int(lstr[1])
    race=int(lstr[2])   
    return(age,sex,race)

In [9]:
l=os.listdir()

l.sort()
l=l[:-4]
lAge=[]
lSex=[]
lRace=[]
labels=[]
for i in range(len(l)):
    age,sex,race=extract_labels_int(l[i])
    lAge = lAge + [age]
    lSex = lSex + [sex]
    lRace = lRace + [race]
    labels = labels + [(age,sex,race)]


In [10]:
def transformer1image(word):
    im = Image.open(word)
    normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    im_format = transforms.Compose([transforms.CenterCrop(224),transforms.ToTensor(), normalize])
    
    q = im_format(im)
    return(q)

def transforBatch(name_list):
    lp=[]
    for i in range(len(name_list)):
        tens=transformer1image(name_list[i])
        lp = lp + [tens]
        llp=torch.stack(lp)
    return(llp)

In [11]:
lImage=l

lImage=np.asarray(lImage)
lAge=np.asarray(lAge)
lSex=np.asarray(lSex)
lRace=np.asarray(lRace)


In [12]:
from sklearn.utils import shuffle

lImage, lAge,lSex,lRace = shuffle(lImage,lAge,lSex,lRace)
batch_size=64
rate=0.75
n=len(lImage)
q=int(rate*n)

lImageTrain=lImage[:q]
lSexTrain=lSex[:q]
lRaceTrain=lRace[:q]
lAgeTrain=lAge[:q]

lImageTest=lImage[q:]
lSexTest=lSex[q:]
lRaceTest=lRace[q:]
lAgeTest=lAge[q:]

n=len(lAgeTrain)
nindice=n//batch_size

In [13]:
import torch.optim

In [14]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,96,kernel_size=7,stride=4),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96,256,kernel_size=5,padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256,384,kernel_size=3,padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2))
        self.fc1 = nn.Linear(384*6*6,512)
        self.fc2 = nn.Linear(512,512)
        self.fc3 = nn.Linear(512,2)
        self.fc4 = nn.Linear(512,106)
        self.fc5 = nn.Linear(512,5)
        

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        #print out.size()
        out = F.dropout(F.relu(self.fc1(out)))
        out = F.dropout(F.relu(self.fc2(out)))
        sex = self.fc3(out)
        age = self.fc4(out)
        race = self.fc5(out)

        return (sex,age,race)

In [15]:
use_gpu = torch.cuda.is_available()

In [40]:
cnn = CNN()
if use_gpu:
    cnn.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(cnn.parameters(),lr=0.001,momentum=0.9)

loss_history = []
num_epochs = 10
train_acc_history = []
val_acc_history = []
epoch_history = []
learning_rate = 0.001
best_val_acc = 0.0

In [41]:
def check_acc(cnn,lSexBatch,lRaceBatch,lAgeBatch,lImageBatch):
    num_correct = 0
    num_sample = 0
    for i in range(len(lSexBatch)):
        images = Variable(images).cuda()
        labels = labels.cuda()
        outputs = cnn(images)
        _,pred = torch.max(outputs.data,1)
        num_sample += labels.size(0)
        num_correct += (pred == labels).sum()
    return float(num_correct)/num_sample

In [42]:
def checkAcc (cnn,imageTest,sexTest,ageTest,raceTest):
    batch_size = 40
    totalSize = len(imageTest)
    iterations = totalSize//40
    tombeJuste = (totalSize%40 != 0)
    sexCount = 0
    accAge = 0
    raceCount = 0
    
    for i in range(iterations+tombeJuste):
        batchImage, batchSex, batchAge, batchRace = imageTest[40*i:40*(i+1)],sexTest[40*i:40*(i+1)],ageTest[40*i:40*(i+1)],raceTest[40*i:40*(i+1)]
        currentBatchSize = len(batchImage)
        lInput = Variable(transforBatch(batchImage))
        sexOutput,ageOutput,raceOutput = cnn(lInput)
        sexpred = torch.argmax(sexOutput, dim=1)
        agepred = torch.argmax(ageOutput, dim=1)
        racepred = torch.argmax(raceOutput, dim=1)
        
        sexArray = np.array([sexpred[i] == lSexBatch[i] for i in range(currentBatchSize)])
        sexCount += np.sum(sexArray)
    
        ageArray = np.array([abs(agepred[i]-lAgeBatch[i]) for i in range(currentBatchSize)])
        accAge += np.sum(ageArray)
    
        raceArray = np.array([racepred[i] == lRaceBatch[i] for i in range(currentBatchSize)])
        raceCount += np.sum(raceArray)

    return(sexCount/totalSize, accAge/totalSize, raceCount/totalSize)

In [ ]:
epochs=2
for epoch in range(epochs):
    permutation= np.random.permutation(nindice)
    for i in range(len(permutation)):

        indice=permutation[i]                        
        lSexBatch=torch.from_numpy(lSexTrain[indice:indice+batch_size]).long()
        lRaceBatch=torch.from_numpy(lRaceTrain[indice:indice+batch_size]).long()
        lAgeBatch=torch.from_numpy(lAgeTrain[indice:indice+batch_size]).long()

        lSexBatch=Variable(lSexBatch)
        lRaceBatch=Variable(lRaceBatch)
        lAgeBatch=Variable(lAgeBatch)

        lImageBatch=lImageTrain[indice:indice+batch_size]

        lInput=transforBatch(lImageBatch)
        lInput=Variable(lInput)
        if use_gpu:
            lInput,lSexBatch,lRaceBatch,lAgeBatch = lInput.cuda(),lSexBatch.cuda(),lRaceBatch.cuda(),lAgeBatch.cuda()

        sexpred,agepred,racepred=cnn(lInput)
        loss1 = criterion(sexpred,lSexBatch)
        loss2 = criterion(racepred,lRaceBatch)
        loss3 = criterion(agepred,lAgeBatch)
        loss=loss1
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 5 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                %(epoch+1, num_epochs, i+1, len(lSexTrain)//50, loss))
            print('Accuracy: ',checkAcc(cnn,lImageBatch,lSexBatch,lAgeBatch,lRaceBatch))
        '''
        if epoch % 10 ==0 or epoch == epochs-1:
            learning_rate = learning_rate * 0.9

            train_acc = checkAcc(cnn,lImageBatch,lSexBatch,lAgeBatch,lRaceBatch)
            train_acc_history.append(train_acc)
            print('Train accuracy for epoch {}: {} '.format(epoch + 1,train_acc))

            val_acc = checkAcc(cnn,lImageTest,lSexTest,lAgeTest,lRaceTest)
            val_acc_history.append(val_acc)
            print('Validation accuracy for epoch {} : {} '.format(epoch + 1,val_acc))
            epoch_history.append(epoch+1)
            plot_performance_curves(train_acc_history,val_acc_history,epoch_history)

            is_best = val_acc > best_val_acc
            best_val_acc = max(val_acc,best_val_acc)
            save_checkpoint(
                {'epoch':epoch+1,
                'state_dict':cnn.state_dict(),
                'best_val_acc':best_val_acc,
                'optimizer':optimizer.state_dict()},is_best)7
        '''

Epoch [1/10], Iter [5/355] Loss: 0.6757
Accuracy:  (0.390625, 24.90625, 0.125)
Epoch [1/10], Iter [10/355] Loss: 0.6773
Accuracy:  (0.578125, 30.40625, 0.234375)
Epoch [1/10], Iter [15/355] Loss: 0.6618
Accuracy:  (0.5625, 24.078125, 0.09375)


In [35]:
path = "C:/Users/ANTOINE-PC/Documents/Deep_learning/Projet/Models/model1"
torch.save(cnn.state_dict(),path)

In [38]:
model = CNN()
model.load_state_dict(torch.load(path))
model.eval()
print(model.state_dict())

OrderedDict([('layer1.0.weight', tensor([[[[-0.0657, -0.0712,  0.0090,  ..., -0.0126,  0.0688,  0.0036],
          [ 0.0634,  0.0735, -0.0711,  ...,  0.0529, -0.0110, -0.0087],
          [ 0.0381,  0.0075,  0.0716,  ..., -0.0591, -0.0390, -0.0674],
          ...,
          [ 0.0490, -0.0515,  0.0530,  ..., -0.0467,  0.0009, -0.0270],
          [ 0.0031, -0.0368,  0.0498,  ..., -0.0612, -0.0484,  0.0649],
          [-0.0477, -0.0305,  0.0350,  ..., -0.0628,  0.0054,  0.0412]],

         [[ 0.0155,  0.0048, -0.0201,  ..., -0.0756,  0.0338, -0.0146],
          [-0.0014, -0.0344, -0.0119,  ..., -0.0315, -0.0191, -0.0389],
          [-0.0679, -0.0825, -0.0011,  ...,  0.0488, -0.0679,  0.0735],
          ...,
          [-0.0017,  0.0258, -0.0852,  ..., -0.0127, -0.0717,  0.0605],
          [ 0.0017, -0.0546,  0.0592,  ...,  0.0306,  0.0199, -0.0474],
          [ 0.0655, -0.0014,  0.0374,  ...,  0.0045, -0.0072, -0.0379]],

         [[ 0.0361,  0.0669, -0.0026,  ..., -0.0267, -0.0453, -0.0770